In [105]:
import sys
import re
sys.path.append(r'../../../')

import torch
import syllables
from model import CharRNN, sample

Load checkpoint from model and generate some text

In [106]:
with open('./checkpoints/rnn (haikus).net', 'rb') as f:
    checkpoint = torch.load(f)


In [107]:
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

Normalize the text, clean up stray spaces, and split properly

In [108]:
haiku = sample(loaded, 75, cuda=True, top_k=5, prime="kitten")
haiku = haiku.replace("\n", " ")
haiku = " ".join(haiku.split())
haiku = haiku.split(" ")
haiku = [word if re.findall("I", word) else word.lower() for word in haiku]
haiku

['kitten',
 "would've",
 'been',
 'hot',
 'stars',
 'with',
 'the',
 'best',
 'are',
 'you',
 'going',
 'at',
 'a',
 'championship',
 'I',
 'ca']

Create generator for iteration & break lines according to haiku syllable structure

In [109]:
haiku_gen = (word for word in haiku)

In [110]:
haiku_with_line_breaks = ""
running_syllable_count = 0
line_number = 0
while line_number < 3:
    word = next(haiku_gen)
    running_syllable_count += syllables.estimate(word)
    if running_syllable_count >= 5 and line_number in (0, 2):
        haiku_with_line_breaks += f"{word}\n"
        line_number += 1
        running_syllable_count = 0
    elif running_syllable_count >= 7 and line_number == 1:
        haiku_with_line_breaks += f"{word}\n"
        line_number += 1
        running_syllable_count = 0
    else:
        haiku_with_line_breaks += f"{word} "



In [111]:
haiku_with_line_breaks

"kitten would've been\nhot stars with the best are\nyou going at a championship\n"